### Summary
- Use this trainer if you need to run models that Unsloth doesn't support
### version 2: 
- Currently the qlora configuration has and tested decoder-only models 


In [1]:
%pip install --upgrade pip
# Restart kernel

  Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q -U torch==2.3.0   # nvidia-cudnn-cu12-8.9.2.26 torch-2.3.0 triton-2.3.0
%pip install -q -U transformers #==4.43.3
%pip install -q -U datasets
%pip install -q -U rich
%pip install -q -U accelerate 
%pip install -q -U peft
%pip install -q -U bitsandbytes 
%pip install -q -U optuna
%pip install -q -U clearml
%pip install -q -U trl 
%pip install -q -U boto3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import os

with open('credentials.json') as f:
    credentials = json.load(f)

# Set environment variables
for key, value in credentials.items():
    if not isinstance(value, str):
        value = json.dumps(value)  # Convert dicts or lists to string
    os.environ[key] = value

huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')
aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_key = os.getenv('AWS_SECRET_KEY')

In [15]:
import torch
torch.cuda.empty_cache()

import subprocess
# Run nvidia-smi command and capture the output
result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
print(result.stdout.decode())

Mon Sep 16 23:50:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.78                 Driver Version: 550.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 NVL                On  |   00000000:01:00.0 Off |                   On |
| N/A   31C    P0             88W /  400W |                  N/A   |     N/A      Default |
|                                         |                        |              Enabled |
+-----------------------------------------+-----

In [3]:
%pip install -q nbconvert
# Import Task from clearml
from clearml import Task
task = Task.init(project_name="HuggingFace Transformers",
    task_name="Multilingual LLMs",
    output_uri=False) # don’t save any of the models to clearml
task.set_parameters_as_dict({ 'save_checkpoints': False })    

Note: you may need to restart the kernel to use updated packages.
ClearML Task: created new task id=d5f3076a63c941869cef8bc31598b5ab
2024-09-16 21:51:11,778 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.sil.hosted.allegro.ai/projects/96152c47119843d7a026576ef08e348f/experiments/d5f3076a63c941869cef8bc31598b5ab/output/log


In [5]:
################### read jsonl file into dataset #################################
import json
from datasets import Dataset, DatasetDict

# Initialize a dictionary to hold the lists for each field
train = {'model_inputs': [], 'completion': []}
val = {'model_inputs': [], 'completion': []}

language = 'hejazi'

# Open the file and read line by line
with open('/root/all_llm_data/'+language+'_train_data.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        # Each line is a complete JSON object
        json_object = json.loads(line)
        # Append each field to the appropriate list
        train['model_inputs'].append(json_object.get('model_inputs', ''))  
        train['completion'].append(json_object.get('completion', ''))  

# Open the file and read line by line
with open('/root/all_llm_data/'+language+'_val_data.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        # Each line is a complete JSON object
        json_object = json.loads(line)
        # Append each field to the appropriate list
        val['model_inputs'].append(json_object.get('model_inputs', ''))  
        val['completion'].append(json_object.get('completion', ''))         

# Convert the dictionary of lists into a `Dataset`
dataset_train = Dataset.from_dict(train)
dataset_val = Dataset.from_dict(val)

# Create a `DatasetDict` for a train/val split
dataset_dict = DatasetDict({'train': dataset_train,'val': dataset_val})

# Print the first entry of the dataset to check its structure
print(dataset_dict['train'][0])
print(dataset_dict['val'][0])

Running inside MIG, Nvidia driver cannot export utilization, pushing fixed value 100


{'model_inputs': 'translate arb_Arab to acw_Arab: فِي الْبَدْءِ خَلَقَ اللهُ السَّمَاوَاتِ وَالأَرْضَ،', 'completion': 'فِي الْبَدْاية خَلَقَ اللهُ السَّمَاوَاتِ وَالأَرْضَ،\r\n'}
{'model_inputs': 'translate arb_Arab to acw_Arab: كَانَ قَدْ تَلَقَّنَ طَرِيقَ الرَّبِّ. فَبَدَأَ يَخْطُبُ بِحَمَاسَةٍ شَدِيدَةٍ، وَيُعَلِّمُ الْحَقَائِقَ الْمُخْتَصَّةَ بِيَسُوعَ تَعْلِيماً صَحِيحاً. وَمَعَ أَنَّهُ لَمْ يَكُنْ يَعْرِفُ سِوَى مَعْمُودِيَّةِ يُوحَنَّا،', 'completion': 'وكان يعرف طريق الرب. وصار يتكلم بحماس شديد، ويعلم الحقايق اللي تخص يسوع تعليم صحيح. ورغم أنو كان يعرف معمودية يوحنا بس،\r\n'}


In [6]:
import torch
import os
import tqdm as notebook_tqdm
from clearml import Task
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, BloomTokenizerFast, BloomForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
max_seq_length = 2048 
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, bfloat16 for Ampere+

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=dtype
)

gtp_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules = ["query_key_vale","dense","sense_h_to_4h","dense_4h_to_h"],
    modules_to_save = ["embed_tokens","lm_head"],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
    use_rslora = True,  # Unsloth turned it off since they use custom code
    loftq_config = None, # Use LoftQ later
)

llama_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
    modules_to_save = ["embed_tokens","lm_head"],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
    use_rslora = True,   
    loftq_config = None, # Use LoftQ later
)

def load_model():
    model_name = "bigscience/bloom-7b1"  
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = BloomForCausalLM.from_pretrained(  # AutoModelForCausalLM
        model_name,
        device_map='auto',
        quantization_config=bnb_config
    )
    return model,tokenizer

In [7]:
prompt = """### Instruction:
{}

### Input:
{}

### Response:
{}"""
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }
pass

def read_data(language,dataset_label):
    # Initialize a dictionary to hold the lists for each field
    dataset_dict = {'input': [], 'output': [], 'instruction': []}
        
    # Open the file and read line by line
    with open('/root/all_llm_data/'+language+'_'+dataset_label+'_data.jsonl', 'r', encoding='utf-8') as file:
        for line in file:
            # Each line is a complete JSON object
            json_object = json.loads(line)
            # Append each field to the appropriate list
            instruction = json_object.get('model_inputs', '')[:30]
            dataset_dict['input'].append(json_object.get('model_inputs', '')[32:]) #remove prompt from input  
            dataset_dict['output'].append(json_object.get('completion', '')[:-2]) #remove \r\n from end of output
            dataset_dict['instruction'].append(instruction)
    
    # Convert the dictionary of lists into a `Dataset`
    dataset = Dataset.from_dict(dataset_dict)

    return dataset.map(formatting_prompts_func, batched = True,)

In [8]:
import torch
import random
import numpy as np
from collections import defaultdict
import transformers   
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTConfig, SFTTrainer

def train(model,tokenizer,dataset_train,dataset_val):

    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset_train,
        formatting_func=formatting_prompts_func, 
        data_collator=collator,
        eval_dataset=dataset_val,
        packing=False, 
        #neftune_noise_alpha = 0.1, 
        args=SFTConfig(
            max_seq_length = max_seq_length,
            dataset_num_proc = 2,
            dataset_text_field = "text",
            per_device_train_batch_size=4, # reduce if loss is too high
            per_device_eval_batch_size=4,  
            gradient_accumulation_steps=4, 
            warmup_steps=20,   
            num_train_epochs=1,
            learning_rate=1e-4, # reduce if loss is too high
            bf16=True,
            logging_steps=10,
            optim="adamw_torch",
            weight_decay=0.01,
            lr_scheduler_type="cosine", 
            output_dir="/root/outputs",
        ),
    )

    #@title Show current memory stats
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
    print(f"{start_gpu_memory} GB of memory reserved.")

    trainer_stats = trainer.train()

    #@title Show final memory and time stats
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory         /max_memory*100, 3)
    lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
    print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
    print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")
    
    return model

In [6]:
import boto3
from botocore.exceptions import ClientError

s3 = boto3.client('s3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
)

def upload_file(file_name, bucket, object_name):
    if object_name is None:
        object_name = file_name
    try:
        s3.upload_file(file_name, bucket, Key=object_name)
    except ClientError as e:
        print(e)
        return False
    print("Success!")
    return True

In [13]:
def save_model(model,tokenizer,language):
   # Option 1: Save loRA adapter only, not the full model
   model.save_pretrained("/root/bloom1_"+language)
   tokenizer.save_pretrained("/root/bloom1_"+language)

   files = [
     'adapter_config.json',
     'adapter_model.safetensors',
     #'config.json',
     #'generation_config.json',
     #'model-00001-of-00002.safetensors',
     #'model-00002-of-00002.safetensors',
     #'model.safetensors.index.json',
     'README.md',
     'special_tokens_map.json',
     'tokenizer_config.json',
     'tokenizer.json']
    
   for file in files:
      print(file)
      upload_file("/root/bloom1_" + language + "/" + file, "silnlp", \
         "MT/experiments/Demo_Crystal/trained_models/bloom1_" + language + \
         "/" + file)

In [14]:
#languages = ['balti','bana','bantawa','borong','gutob_gadaba','hejazi','kisar','konda_dora','kuvi','kwaraae','limbum','mbugwe','naxi','rajbanshi','siddi','tai_nua','waima','western_chawma']

languages = ['bantawa','tai_nua']

for language in languages:
    torch.cuda.empty_cache()
    print(f"Processing language: {language}")

    print("Loading model")
    model,tokenizer = load_model()
    EOS_TOKEN = tokenizer.eos_token or "<|endoftext|>" 

    print("Reading training data")
    dataset_train = read_data(language,"train")

    print("Reading validation data")
    dataset_val = read_data(language,"val")

    if language != "mbugwe": # no test data for mbugwe
        print("Reading test data")
        dataset_test = read_data(language,"test")

    print("Training")
    # Training setup 
    collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # causal language modeling

    model.gradient_checkpointing_enable()    # Memory-efficient training
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, gtp_config)   # llama_config, gtp_config 

    model = train(model,tokenizer,dataset_train, dataset_val)

    print(f"Saving model adapter for {language}")
    save_model(model,tokenizer,language)

Processing language: bantawa
Loading model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Reading training data


Map:   0%|          | 0/7687 [00:00<?, ? examples/s]

Reading validation data


...

/root/.clearml/venvs-builds/3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


GPU = NVIDIA H100 NVL MIG 3g.47gb. Max memory = 46.375 GB.
44.529 GB of memory reserved.
2024-09-16 23:14:13,087 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers_3/accelerator_config[AcceleratorConfig])


/root/.clearml/venvs-builds/3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "../c10/cuda/CUDACachingAllocator.cpp":844, please report a bug to PyTorch. 